<a href="https://colab.research.google.com/github/hayleypc/HawaiiClimate/blob/main/soil_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import geopandas as gpd

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score

def preprocess_and_train_model(matched_data):
    # Data Preprocessing Steps
    # matched_data = matches_within_distance.groupby('unique_id').sample(n=1)
    # matched_data = matched_data.reset_index(drop=True)
    matched_data['distance'] = 0
    unique_rows = matched_data[matched_data["depth_adj_bottom"] == '20']
    unique_rows = unique_rows.drop_duplicates(subset=['latitude', 'longitude'])
    unique_rows = unique_rows[unique_rows['distance'] < 251] # remove set that didn't make sense (ocean point maybe)
    matched_data = unique_rows
    matched_data['imp_c_float'] = [float(datum) for datum in matched_data['imp_c']]
    id_fields = matched_data[['source_dataset', 'island', 'soil_column_id', 'unique_id', 'depth_top', 'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude']]
    keep_cols = [21,22,23,24,25,26,27,28, 29, 31,32, 33,34,36,36,37,38,39,40,41,42,43,44,45,46,47, 48, 55]
    numeric_cols = matched_data.iloc[:,keep_cols]
    numeric_cols.replace('', np.nan, inplace=True)
    numeric_cols = numeric_cols.astype(float)
    numeric_cols.fillna(0, inplace=True)
    scaler = MinMaxScaler()
    scaler.fit(numeric_cols)
    scaled_numeric_cols = scaler.transform(numeric_cols)
    scaled_numeric_df = pd.DataFrame(scaled_numeric_cols, columns=numeric_cols.columns, index=numeric_cols.index)
    numeric_df = pd.concat([id_fields, scaled_numeric_df], axis=1)

    # Model Definition
    def build_model(input_shape):
        model = Sequential([
            Dense(256, activation='relu', input_dim=input_shape),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(512, activation='relu'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(1, activation='linear')  # Adjust output layer according to your needs
        ])
        return model

    X = scaled_numeric_df.iloc[:, :-1]
    y = scaled_numeric_df.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = build_model(X_train.shape[1])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=128, verbose=1)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')
    model.fit(X_train, y_train, validation_split=0.2, epochs=300, batch_size=128, verbose=1)
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error')
    model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=1)

    test_loss = model.evaluate(X_test, y_test)

    # Predictions and Evaluation
    predictions = model.predict(X_test).flatten()
    dummy_array = np.full((len(predictions), 28), fill_value=0.5)  # Example for reverse scaling
    dummy_array[:, -1] = predictions
    inversed_predictions = scaler.inverse_transform(dummy_array)[:, -1]
    dummy_array[:, -1] = y_test
    inversed_truth = scaler.inverse_transform(dummy_array)[:, -1]
    r_squared = r2_score(inversed_truth, inversed_predictions)

    return model, test_loss, r_squared, inversed_predictions, scaler

# Use the function with your DataFrame
# model, test_loss, r_squared, predictions = preprocess_and_train_model(your_dataframe)


In [5]:
def predict_on_reserve(matched_data,model,scaler):
    # Data Preprocessing Steps
    # matched_data = matches_within_distance.groupby('unique_id').sample(n=1)
    # matched_data = matched_data.reset_index(drop=True)
    matched_data['distance'] = 0
    unique_rows = matched_data[matched_data["depth_adj_bottom"] == '20']
    unique_rows = unique_rows.drop_duplicates(subset=['latitude', 'longitude'])
    unique_rows = unique_rows[unique_rows['distance'] < 251]
    matched_data = unique_rows
    matched_data['imp_c_float'] = [float(datum) for datum in matched_data['imp_c']]
    id_fields = matched_data[['source_dataset', 'island', 'soil_column_id', 'unique_id', 'depth_top', 'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude']]
    keep_cols = [21,22,23,24,25,26,27,28, 29, 31,32, 33,34,36,36,37,38,39,40,41,42,43,44,45,46,47, 48, 55]
    numeric_cols = matched_data.iloc[:,keep_cols]
    numeric_cols.replace('', np.nan, inplace=True)
    numeric_cols = numeric_cols.astype(float)
    numeric_cols.fillna(0, inplace=True)
    scaled_numeric_cols = scaler.transform(numeric_cols)
    scaled_numeric_df = pd.DataFrame(scaled_numeric_cols, columns=numeric_cols.columns, index=numeric_cols.index)
    numeric_df = pd.concat([id_fields, scaled_numeric_df], axis=1)


    X = scaled_numeric_df.iloc[:, :-1]
    y = scaled_numeric_df.iloc[:, -1]

    # Predictions and Evaluation
    predictions = model.predict(X).flatten()
    dummy_array = np.full((len(predictions), 28), fill_value=0.5)  # Example for reverse scaling
    dummy_array[:, -1] = predictions
    inversed_predictions = scaler.inverse_transform(dummy_array)[:, -1]
    dummy_array[:, -1] = y
    inversed_truth = scaler.inverse_transform(dummy_array)[:, -1]
    matched_data['inversed_predictions'] = inversed_predictions.flatten()
    matched_data['inversed_imp_c'] =inversed_truth.flatten()
    return matched_data

In [6]:
file_path = '/content/drive/My Drive/sequence_data.csv'

In [7]:
drivers_gpd = gpd.read_file('/content/drive/MyDrive/hawaii_soils/Analysis Data/250_summary_grid_dt.gpkg')

In [8]:
soils_csv = gpd.read_file('/content/drive/MyDrive/hawaii_soils/HI soils data/combined_soc_2024_04_05.csv')

In [9]:
from shapely.geometry import Point

soils_csv = soils_csv[(soils_csv['latitude'] != '') & (soils_csv['longitude'] != '')]
soils_csv['geometry'] = soils_csv.apply(lambda row: Point(float(row['longitude']), float(row['latitude'] )), axis=1)
soils_gpd = gpd.GeoDataFrame(soils_csv, geometry='geometry', crs="EPSG:4326")

In [10]:


# Ensure both GeoDataFrames have the same CRS
soils_gpd = soils_gpd.to_crs(drivers_gpd.crs)

# Perform spatial join
matched_data = gpd.sjoin_nearest(soils_gpd, drivers_gpd, how='left', distance_col='distance')

In [11]:
# Apply a buffer to each geometry in one of the GeoDataFrames (e.g., soils_gpd)
soils_buffered = soils_gpd.copy()

soils_buffered.geometry = soils_buffered.to_crs(epsg=32604).geometry.buffer(1000).to_crs('ESRI:102261')

# soils_gpd = soils_gpd
# Step 2: Spatial Join
# Perform a spatial join with the buffered GeoDataFrame
# This finds all drivers_gpd points that fall within the 10,000-meter buffer of any point in soils_gpd
matches_within_distance = gpd.sjoin(soils_buffered, drivers_gpd, how='left', op='intersects')


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [34]:
# Use the function with your DataFrame
reserve_data = matches_within_distance.groupby('unique_id').sample(n=1)


prediction_list = []

for i in range(10):
  matched_data = matches_within_distance.groupby('unique_id').sample(n=1)
  matched_data = matched_data.reset_index(drop=True)
  model, test_loss, r_squared, inversed_predictions, scaler = preprocess_and_train_model(matched_data)

  predictions = predict_on_reserve(reserve_data,model,scaler)
  prediction_list.append(predictions)


<ipython-input-4-352fa283bc47>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


Epoch 1/100
5/5 [==============================] - 2s 75ms/step - loss: 2.7220 - val_loss: 0.0221
Epoch 2/100
5/5 [==============================] - 0s 25ms/step - loss: 1.9693 - val_loss: 0.0238
Epoch 3/100
5/5 [==============================] - 0s 25ms/step - loss: 1.9821 - val_loss: 0.0259
Epoch 4/100
5/5 [==============================] - 0s 25ms/step - loss: 2.1589 - val_loss: 0.0264
Epoch 5/100
5/5 [==============================] - 0s 25ms/step - loss: 1.7274 - val_loss: 0.0263
Epoch 6/100
5/5 [==============================] - 0s 24ms/step - loss: 1.9915 - val_loss: 0.0334
Epoch 7/100
5/5 [==============================] - 0s 25ms/step - loss: 1.8019 - val_loss: 0.0320
Epoch 8/100
5/5 [==============================] - 0s 25ms/step - loss: 1.8450 - val_loss: 0.0324
Epoch 9/100
5/5 [==============================] - 0s 28ms/step - loss: 1.2920 - val_loss: 0.0339
Epoch 10/100
5/5 [==============================] - 0s 24ms/step - loss: 1.1653 - val_loss: 0.0310
Epoch 11/100
5/5 [=

<ipython-input-5-8459f0b8cc1d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)
<ipython-input-4-352fa283bc47>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


Epoch 1/100
5/5 [==============================] - 2s 71ms/step - loss: 2.7955 - val_loss: 0.0207
Epoch 2/100
5/5 [==============================] - 0s 24ms/step - loss: 2.6589 - val_loss: 0.0422
Epoch 3/100
5/5 [==============================] - 0s 29ms/step - loss: 2.2615 - val_loss: 0.0242
Epoch 4/100
5/5 [==============================] - 0s 25ms/step - loss: 1.9404 - val_loss: 0.0282
Epoch 5/100
5/5 [==============================] - 0s 25ms/step - loss: 1.7248 - val_loss: 0.0246
Epoch 6/100
5/5 [==============================] - 0s 24ms/step - loss: 1.5898 - val_loss: 0.0262
Epoch 7/100
5/5 [==============================] - 0s 27ms/step - loss: 1.6980 - val_loss: 0.0272
Epoch 8/100
5/5 [==============================] - 0s 23ms/step - loss: 1.5939 - val_loss: 0.0272
Epoch 9/100
5/5 [==============================] - 0s 24ms/step - loss: 1.1108 - val_loss: 0.0273
Epoch 10/100
5/5 [==============================] - 0s 24ms/step - loss: 1.3128 - val_loss: 0.0277
Epoch 11/100
5/5 [=

<ipython-input-5-8459f0b8cc1d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


...

<ipython-input-5-8459f0b8cc1d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)
<ipython-input-4-352fa283bc47>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


Epoch 1/100
5/5 [==============================] - 2s 65ms/step - loss: 2.3072 - val_loss: 0.0201
Epoch 2/100
5/5 [==============================] - 0s 22ms/step - loss: 2.3834 - val_loss: 0.0241
Epoch 3/100
5/5 [==============================] - 0s 23ms/step - loss: 2.0011 - val_loss: 0.0225
Epoch 4/100
5/5 [==============================] - 0s 25ms/step - loss: 2.1287 - val_loss: 0.0362
Epoch 5/100
5/5 [==============================] - 0s 22ms/step - loss: 1.5632 - val_loss: 0.0431
Epoch 6/100
5/5 [==============================] - 0s 22ms/step - loss: 2.2534 - val_loss: 0.0330
Epoch 7/100
5/5 [==============================] - 0s 23ms/step - loss: 1.8866 - val_loss: 0.0325
Epoch 8/100
5/5 [==============================] - 0s 22ms/step - loss: 2.1876 - val_loss: 0.0333
Epoch 9/100
5/5 [==============================] - 0s 24ms/step - loss: 1.5411 - val_loss: 0.0379
Epoch 10/100
5/5 [==============================] - 0s 23ms/step - loss: 1.6113 - val_loss: 0.0356
Epoch 11/100
5/5 [=

<ipython-input-5-8459f0b8cc1d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)
<ipython-input-4-352fa283bc47>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


Epoch 1/100
5/5 [==============================] - 2s 67ms/step - loss: 2.0631 - val_loss: 0.0425
Epoch 2/100
5/5 [==============================] - 0s 23ms/step - loss: 2.4133 - val_loss: 0.0317
Epoch 3/100
5/5 [==============================] - 0s 23ms/step - loss: 2.2873 - val_loss: 0.0424
Epoch 4/100
5/5 [==============================] - 0s 26ms/step - loss: 2.6273 - val_loss: 0.0243
Epoch 5/100
5/5 [==============================] - 0s 24ms/step - loss: 2.4891 - val_loss: 0.0303
Epoch 6/100
5/5 [==============================] - 0s 23ms/step - loss: 1.6788 - val_loss: 0.0337
Epoch 7/100
5/5 [==============================] - 0s 23ms/step - loss: 1.4753 - val_loss: 0.0320
Epoch 8/100
5/5 [==============================] - 0s 25ms/step - loss: 1.3087 - val_loss: 0.0322
Epoch 9/100
5/5 [==============================] - 0s 25ms/step - loss: 1.3667 - val_loss: 0.0300
Epoch 10/100
5/5 [==============================] - 0s 24ms/step - loss: 1.0537 - val_loss: 0.0292
Epoch 11/100
5/5 [=

<ipython-input-5-8459f0b8cc1d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


In [35]:
combined_array = np.array([predictions['inversed_predictions'].values for predictions in prediction_list])

In [36]:
combined_array[combined_array < 0 ] = 0

In [37]:
arr_min = np.min(combined_array.flatten())
arr_max = np.max(combined_array.flatten())

In [38]:
dist_array = [np.sort(np.array([i[j] for  i in combined_array])) for j in range(combined_array.shape[1])]
norm_dist_array =  [np.sort((np.array([i[j] for  i in combined_array]) - arr_min) / (arr_max-arr_min)) for j in range(combined_array.shape[1])]

In [46]:
len(norm_dist_array)

849

In [40]:
# array = norm_dist_array[0]
# # Generating y values as indices (percentiles in this context)
# y_values = np.linspace(1, 100, len(array))

# # Plotting
# plt.figure(figsize=(10, 6))
# plt.plot(array, y_values, marker='o', linestyle='-', color='b')
# plt.title('Index (Percentile) by Value')
# plt.xlabel('Value')
# plt.ylabel('Percentile')
# plt.grid(True)
# plt.show()

# # Plotting
# plt.figure(figsize=(10, 6))
# plt.hist(array, bins=len(array), edgecolor='black')
# plt.title('Histogram of Values')
# plt.xlabel('Value')
# plt.ylabel('Frequency')
# plt.grid(True)
# plt.show()



In [49]:
matched_data =  reserve_data

In [52]:
matched_data['distance'] = 0
unique_rows = matched_data[matched_data["depth_adj_bottom"] == '20']
unique_rows = unique_rows.drop_duplicates(subset=['latitude', 'longitude'])
unique_rows = unique_rows[unique_rows['distance'] < 251]
matched_data = unique_rows
matched_data['imp_c_float'] = [float(datum) for datum in matched_data['imp_c']]
id_fields = matched_data[['source_dataset', 'island', 'soil_column_id', 'unique_id', 'depth_top', 'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude']]
keep_cols = [21,22,23,24,25,26,27,28, 29, 31,32, 33,34,36,36,37,38,39,40,41,42,43,44,45,46,47, 48, 55]
numeric_cols = matched_data.iloc[:,keep_cols]
numeric_cols.replace('', np.nan, inplace=True)
numeric_cols = numeric_cols.astype(float)
numeric_cols.fillna(0, inplace=True)
scaler = MinMaxScaler()
scaler.fit(numeric_cols)
scaled_numeric_cols = scaler.transform(numeric_cols)
scaled_numeric_df = pd.DataFrame(scaled_numeric_cols, columns=numeric_cols.columns, index=numeric_cols.index)
numeric_df = pd.concat([id_fields, scaled_numeric_df], axis=1)

<ipython-input-52-4384be5e59d6>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


In [54]:
# numeric_df.columns

In [55]:
# Model Definition
def build_model(input_shape,output_shape):
    model = Sequential([
        Dense(256, activation='relu', input_dim=input_shape),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(512, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(output_shape, activation='linear')  # Adjust output layer according to your needs
    ])
    return model


In [58]:
scaled_numeric_df['norm_dist_array'] = norm_dist_array

In [59]:
scaled_numeric_df.dropna(inplace=True)

In [60]:
norm_dist_array = np.array([i for i in scaled_numeric_df['norm_dist_array']])

In [61]:

X = scaled_numeric_df.iloc[:, :-1]
y = scaled_numeric_df.iloc[:, -1]

x = np.array(X)
y = np.array(norm_dist_array)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [62]:
# print(X_train.shape[1])
model = build_model(X_train.shape[1],output_shape=10)

In [63]:
y_train.shape

(679, 10)

In [ ]:


# model = build_model((X_train.shape[1]))
# model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error',metrics = ['mae'])
# model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=128, verbose=1)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error',metrics = ['mae'])
model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=1)
model.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error',metrics = ['mae'])
model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=1)

test_loss = model.evaluate(X_test, y_test)

# Predictions and Evaluation
predictions = model.predict(X_test)
# dummy_array = np.full((len(predictions), 28), fill_value=0.5)  # Example for reverse scaling
# dummy_array[:, -1] = predictions
# inversed_predictions = scaler.inverse_transform(dummy_array)[:, -1]
# dummy_array[:, -1] = y_test
# inversed_truth = scaler.inverse_transform(dummy_array)[:, -1]
# r_squared = r2_score(inversed_truth, inversed_predictions)

# return model, test_loss, r_squared, inversed_predictions, scaler

Epoch 1/600
5/5 [==============================] - 2s 71ms/step - loss: 2.3152 - mae: 1.1052 - val_loss: 0.0716 - val_mae: 0.2118
Epoch 2/600
5/5 [==============================] - 0s 30ms/step - loss: 2.0177 - mae: 1.0503 - val_loss: 0.0684 - val_mae: 0.2052
Epoch 3/600
5/5 [==============================] - 0s 29ms/step - loss: 1.8828 - mae: 0.9991 - val_loss: 0.0677 - val_mae: 0.2035
Epoch 4/600
5/5 [==============================] - 0s 24ms/step - loss: 1.7238 - mae: 0.9822 - val_loss: 0.0681 - val_mae: 0.2045
Epoch 5/600
5/5 [==============================] - 0s 25ms/step - loss: 1.5920 - mae: 0.9396 - val_loss: 0.0694 - val_mae: 0.2072
Epoch 6/600
5/5 [==============================] - 0s 23ms/step - loss: 1.6022 - mae: 0.9320 - val_loss: 0.0707 - val_mae: 0.2101
Epoch 7/600
5/5 [==============================] - 0s 24ms/step - loss: 1.5484 - mae: 0.9144 - val_loss: 0.0718 - val_mae: 0.2124
Epoch 8/600
5/5 [==============================] - 0s 21ms/step - loss: 1.4788 - mae: 0.89

In [ ]:
i = 4
for i in range(5):
  print(predictions[i])
  print(y_test[i])